In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [2]:
pd.options.display.max_columns = 999
sns.set_style('whitegrid')

# https://metadata.phila.gov/#home/datasetdetails/5543865f20583086178c4ee5/representationdetails/55d624fdad35c7e854cb21a4/

In [3]:
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

from sklearn.preprocessing import RobustScaler, OneHotEncoder

from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

In [4]:
df = pd.read_csv('PHL_Building_Dataset_ML_Regression.csv')
df = df.drop(columns=['Unnamed: 0'])
df

,basements,building_code_description,central_air,depth,exterior_condition,fireplaces,frontage,garage_type,interior_condition,market_value,number_of_bathrooms,number_of_bedrooms,number_of_rooms,number_stories,parcel_shape,street_designation,topography,total_area,total_livable_area,type_heater,view_type,year_built,zoning,building_description,segment
0,0,ROW B/GAR 2 STY MASONRY,Y,90.00,4.0,0.0,29.17,A,4.0,257500.0,2.0,3.0,6.0,2.0,E,ST,6,2625.30,1266.0,H,I,1960.0,1,MASONRY,2
1,0,ROW B/GAR 2 STY MASONRY,Y,90.00,4.0,0.0,18.03,A,4.0,249400.0,2.0,3.0,6.0,2.0,E,ST,6,1622.70,1266.0,A,I,1960.0,1,MASONRY,2
2,0,ROW B/GAR 2 STY MASONRY,Y,90.00,4.0,0.0,18.05,A,4.0,249500.0,2.0,3.0,6.0,2.0,E,ST,6,1624.50,1266.0,A,I,1960.0,1,MASONRY,2
3,0,ROW B/GAR 2 STY MASONRY,Y,90.00,4.0,0.0,18.08,A,4.0,249500.0,2.0,3.0,6.0,2.0,E,ST,6,1627.20,1266.0,H,I,1960.0,1,MASONRY,2
4,0,ROW B/GAR 2 STY MASONRY,Y,90.00,4.0,0.0,18.71,A,4.0,253800.0,2.0,3.0,6.0,2.0,E,ST,6,1683.90,1310.0,H,I,1960.0,1,MASONRY,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44479,D,ROW 3 STY MASONRY,N,100.00,4.0,0.0,21.87,0,4.0,117500.0,0.0,3.0,6.0,3.0,A,ST,6,1651.00,2907.0,H,I,1929.0,1,MASONRY,1
44480,D,ROW 2 STY MASONRY,Y,102.19,3.0,0.0,18.24,0,3.0,70800.0,2.0,3.0,9.0,3.0,A,ST,6,1811.00,2026.0,A,I,2005.0,1,MASONRY,1
44481,D,ROW 3 STY MASONRY,Y,103.26,3.0,0.0,16.00,0,3.0,23600.0,1.0,3.0,6.0,3.0,A,ST,6,1604.00,2082.0,A,I,2005.0,1,MASONRY,1
44482,D,ROW 3 STY MASONRY,Y,90.00,3.0,0.0,16.00,0,3.0,23600.0,1.0,3.0,6.0,3.0,E,ST,6,1440.00,1387.0,A,I,2005.0,1,MASONRY,1


In [5]:
# Rename Segment
df['segment'] = df['segment'].replace({1:'Bottom', 2:'Lower Middle', 3:'Middle', 4:'Upper Middle', 5:'Top'})

# Splitting Data

In [6]:
X = df.drop(columns=['building_code_description', 'market_value'])
y = df['market_value']

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

# Evaluation Matrix

In [8]:
# Evaluation Matrix Regression
def Eva_Matrix(Model, X_train, y_train, X_test, y_test, Name):
    y_pred_train = Model.predict(X_train)
    r2_train = r2_score(y_train, y_pred_train)
    mae_train = mean_absolute_error(y_train, y_pred_train)
    mse_train = mean_squared_error(y_train, y_pred_train)
    rmse_train = np.sqrt(mse_train)
    y_pred_test = Model.predict(X_test)
    r2_test = r2_score(y_test, y_pred_test)
    mae_test = mean_absolute_error(y_test, y_pred_test)
    mse_test = mean_squared_error(y_test, y_pred_test)
    rmse_test = np.sqrt(mse_test)
    data = {
        f"Training {Name}" : [r2_train, mae_train, mse_train, rmse_train],
        f"Test {Name}" : [r2_test, mae_test, mse_test, rmse_test]
    }
    df = pd.DataFrame(data=data, index=['R2', 'MAE', 'MSE', 'RMSE'])
    return df

# Pipeline

In [298]:
num_columns = ['fireplaces', 'number_of_rooms', 'number_stories', 'total_area', 
               'total_livable_area']

cat_columns = ['basements', 'central_air', 'exterior_condition', 'garage_type',
               'interior_condition', 'parcel_shape', 'street_designation', 'topography',
               'type_heater', 'view_type', 'building_description']

numeric_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', RobustScaler()),
    #('poly', PolynomialFeatures(degree=3, include_bias=False)),
    #('power', PowerTransformer(method='yeo-johnson'))
])

categoric_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder())
])

preprocessor = ColumnTransformer([
    ('numeric', numeric_pipeline, num_columns),
    ('categoric', categoric_pipeline, cat_columns)
])

pipeSVM = Pipeline([
    ("prep", preprocessor),
    ("algo", SVR(max_iter=400))
])

pipeLR = Pipeline([
    ("prep", preprocessor),
    ("algo", LinearRegression())
])

pipeLasso = Pipeline([
    ("prep", preprocessor),
    ("algo", Lasso())
])

pipeRidge = Pipeline([
    ("prep", preprocessor),
    ("algo", Ridge())
])

pipeElasticNet = Pipeline([
    ("prep", preprocessor),
    ("algo", ElasticNet())
])

pipeKNN = Pipeline([
    ("prep", preprocessor),
    ("algo", KNeighborsRegressor())
])
               
pipeDT = Pipeline([
    ("prep", preprocessor),
    ("algo", DecisionTreeRegressor())
])
            
pipeRF = Pipeline([
    ("prep", preprocessor),
    ("algo", RandomForestRegressor())
])

# Base Model KNN

In [10]:
pipeKNN.fit(X_train, y_train)

Pipeline(steps=[('prep',
                 ColumnTransformer(transformers=[('numeric',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer())]),
                                                  ['fireplaces',
                                                   'number_of_rooms',
                                                   'number_stories',
                                                   'total_area',
                                                   'total_livable_area']),
                                                 ('categoric',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('encoder',
                                                                   OneHotEnco

In [11]:
y_KNN_test = pipeKNN.predict(X_test)

In [12]:
df_KNN = Eva_Matrix(pipeKNN, X_train, y_train, X_test, y_test, "Pipeline KNN")
df_KNN

,Training Pipeline KNN,Test Pipeline KNN
R2,7.648403e-01,6.382117e-01
MAE,3.103323e+04,4.092095e+04
MSE,4.277652e+09,7.529401e+09
RMSE,6.540376e+04,8.677212e+04


# Base Model Linear Regression

In [13]:
pipeLR.fit(X_train, y_train)

Pipeline(steps=[('prep',
                 ColumnTransformer(transformers=[('numeric',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer())]),
                                                  ['fireplaces',
                                                   'number_of_rooms',
                                                   'number_stories',
                                                   'total_area',
                                                   'total_livable_area']),
                                                 ('categoric',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('encoder',
                                                                   OneHotEnco

In [14]:
y_LR_test = pipeLR.predict(X_test)

In [15]:
df_LR = Eva_Matrix(pipeLR, X_train, y_train, X_test, y_test, "Pipeline LR")
df_LR

,Training Pipeline LR,Test Pipeline LR
R2,7.518020e-01,7.528840e-01
MAE,4.124612e+04,4.254337e+04
MSE,4.514825e+09,5.142883e+09
RMSE,6.719245e+04,7.171390e+04


# Base Model Ridge

In [16]:
pipeRidge.fit(X_train, y_train)

Pipeline(steps=[('prep',
                 ColumnTransformer(transformers=[('numeric',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer())]),
                                                  ['fireplaces',
                                                   'number_of_rooms',
                                                   'number_stories',
                                                   'total_area',
                                                   'total_livable_area']),
                                                 ('categoric',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('encoder',
                                                                   OneHotEnco

In [17]:
y_Ridge_test = pipeRidge.predict(X_test)

In [18]:
df_Ridge = Eva_Matrix(pipeRidge, X_train, y_train, X_test, y_test, "Pipeline Ridge")
df_Ridge

,Training Pipeline Ridge,Test Pipeline Ridge
R2,6.237616e-01,6.369530e-01
MAE,5.384231e+04,5.541533e+04
MSE,6.843932e+09,7.555597e+09
RMSE,8.272806e+04,8.692293e+04


# Base Model Lasso

In [19]:
pipeLasso.fit(X_train, y_train)

Pipeline(steps=[('prep',
                 ColumnTransformer(transformers=[('numeric',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer())]),
                                                  ['fireplaces',
                                                   'number_of_rooms',
                                                   'number_stories',
                                                   'total_area',
                                                   'total_livable_area']),
                                                 ('categoric',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('encoder',
                                                                   OneHotEnco

In [20]:
y_Lasso_test = pipeLasso.predict(X_test)

In [21]:
df_Lasso = Eva_Matrix(pipeLasso, X_train, y_train, X_test, y_test, "Pipeline Lasso")
df_Lasso

,Training Pipeline Lasso,Test Pipeline Lasso
R2,7.518380e-01,7.531919e-01
MAE,4.124244e+04,4.251425e+04
MSE,4.514169e+09,5.136477e+09
RMSE,6.718757e+04,7.166922e+04


# Base Model ElasticNet

In [22]:
pipeElasticNet.fit(X_train, y_train)

Pipeline(steps=[('prep',
                 ColumnTransformer(transformers=[('numeric',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer())]),
                                                  ['fireplaces',
                                                   'number_of_rooms',
                                                   'number_stories',
                                                   'total_area',
                                                   'total_livable_area']),
                                                 ('categoric',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('encoder',
                                                                   OneHotEnco

In [23]:
y_ElasticNet_test = pipeElasticNet.predict(X_test)

In [24]:
df_ElasticNet = Eva_Matrix(pipeElasticNet, X_train, y_train, X_test, y_test, "Pipeline ElasticNet")
df_ElasticNet

,Training Pipeline ElasticNet,Test Pipeline ElasticNet
R2,7.014703e-01,7.073878e-01
MAE,4.487003e+04,4.640047e+04
MSE,5.430379e+09,6.089734e+09
RMSE,7.369110e+04,7.803675e+04


# Base Model SVM

In [25]:
pipeSVM.fit(X_train, y_train)

Pipeline(steps=[('prep',
                 ColumnTransformer(transformers=[('numeric',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer())]),
                                                  ['fireplaces',
                                                   'number_of_rooms',
                                                   'number_stories',
                                                   'total_area',
                                                   'total_livable_area']),
                                                 ('categoric',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('encoder',
                                                                   OneHotEnco

In [26]:
y_SVM_test = pipeSVM.predict(X_test)

In [27]:
df_SVM = Eva_Matrix(pipeSVM, X_train, y_train, X_test, y_test, "Pipeline SVM")
df_SVM

,Training Pipeline SVM,Test Pipeline SVM
R2,-2.141573e+00,-1.832491e+00
MAE,2.201628e+05,2.216299e+05
MSE,5.714651e+10,5.894872e+10
RMSE,2.390534e+05,2.427936e+05


# Base Model DecisionTreeRegressor

In [32]:
pipeDT.fit(X_train, y_train)

Pipeline(steps=[('prep',
                 ColumnTransformer(transformers=[('numeric',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer())]),
                                                  ['fireplaces',
                                                   'number_of_rooms',
                                                   'number_stories',
                                                   'total_area',
                                                   'total_livable_area']),
                                                 ('categoric',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('encoder',
                                                                   OneHotEnco

In [33]:
y_DT_test = pipeDT.predict(X_test)

In [41]:
df_DT = Eva_Matrix(pipeDT, X_train, y_train, X_test, y_test, "Pipeline DT")
df_DT

,Training Pipeline DT,Test Pipeline DT
R2,9.997697e-01,6.868985e-01
MAE,2.627620e+02,3.111364e+04
MSE,4.190052e+06,6.516150e+09
RMSE,2.046962e+03,8.072267e+04


# Base Model RandomForestRegressor

In [35]:
pipeRF.fit(X_train, y_train)

Pipeline(steps=[('prep',
                 ColumnTransformer(transformers=[('numeric',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer())]),
                                                  ['fireplaces',
                                                   'number_of_rooms',
                                                   'number_stories',
                                                   'total_area',
                                                   'total_livable_area']),
                                                 ('categoric',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('encoder',
                                                                   OneHotEnco

In [36]:
y_RF_test = pipeRF.predict(X_test)

In [42]:
df_RF = Eva_Matrix(pipeRF, X_train, y_train, X_test, y_test, "Pipeline RF")
df_RF

,Training Pipeline RF,Test Pipeline RF
R2,9.773417e-01,8.406967e-01
MAE,8.468079e+03,2.384308e+04
MSE,4.121646e+08,3.315360e+09
RMSE,2.030184e+04,5.757917e+04


# Fine Tuning

## DecisionTreeRegressor

In [319]:
pipeDT_Tuning = Pipeline([
    ("prep", preprocessor),
    ("algo", DecisionTreeRegressor(max_depth = None, max_features = 0.9, min_samples_leaf = 25, min_samples_split = 24, random_state=42))
])

In [320]:
pipeDT_Tuning.fit(X_train, y_train)

Pipeline(steps=[('prep',
                 ColumnTransformer(transformers=[('numeric',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   RobustScaler())]),
                                                  ['fireplaces',
                                                   'number_of_rooms',
                                                   'number_stories',
                                                   'total_area',
                                                   'total_livable_area']),
                                                 ('categoric',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequen

In [321]:
y_DT_Fine_Tuned_test = pipeDT_Tuning.predict(X_test)

In [322]:
df_DT_Fine_Tuned = Eva_Matrix(pipeDT_Tuning, X_train, y_train, X_test, y_test, "Pipeline DT Tuned")
df_DT_Fine_Tuned

,Training Pipeline DT Tuned,Test Pipeline DT Tuned
R2,8.071473e-01,7.769997e-01
MAE,2.931071e+04,3.345081e+04
MSE,3.508071e+09,4.640997e+09
RMSE,5.922897e+04,6.812486e+04


In [ ]:
bikin residual plot -> untuk yg test, kalo mau buat yg train juga boleh

In [ ]:
error = abs(y_DT_Fine_Tuned_test - ytest)

In [ ]:
dftest = pd.concat(x_test,y_test)

In [ ]:
dftest['error'] = error

In [ ]:
bikin histogram dari dftest error

In [ ]:
scatter plot antara dftest error & dftest y_test

In [ ]:
nanti keliatan bentuknya gimana